In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import tensorflow as tf
import tensorflow_probability as tfp
import unittest

from stat570 import datasets

failure_stresses = datasets.failure_stresses.load_data()
failure_stresses

,0,1,2,3,4,5,6,7,8,9,10,11,12
Length (mm),,,,,,,,,,,,,
1,2.247,2.640,2.842,2.908,3.099,3.126,3.245,3.328,3.355,3.383,3.572,3.581,3.681
10,1.901,2.132,2.203,2.228,2.257,2.350,2.361,2.396,2.397,2.445,2.454,2.454,2.474
20,1.312,1.314,1.479,1.552,1.700,1.803,1.861,1.865,1.944,1.958,1.966,1.997,2.006
50,1.339,1.434,1.549,1.574,1.589,1.613,1.746,1.753,1.764,1.807,1.812,1.840,1.852


## Choosing Prior Hyperparameters

In [3]:
def compute_log_norm_prior(l, u, density):
    """Computes the hyperparameters for lognormal distribution.
    
    Let X = exp(mu + sigma*Z). Computes mu and sigma such that
    P(l <= X <= u) = density.
    """
    tail_density = (1 - density)/2
    a, b = stats.norm.ppf(tail_density), stats.norm.ppf(1 - tail_density)    
    sigma = np.log(u/l)/(b - a)
    mu = np.log(u) - b*sigma
    return mu, sigma

class ComputeLogNormPrior(unittest.TestCase):    
    def test_compute_log_norm_prior(self):
        l, u = stats.uniform.rvs(0, 10), stats.uniform.rvs(20, 30)
        mu, sigma = compute_log_norm_prior(l, u, 0.9)
        self.assertAlmostEqual(
            stats.lognorm.cdf(l, scale=np.exp(mu), s=sigma), 0.05)
        self.assertAlmostEqual(
            stats.lognorm.sf(u, scale=np.exp(mu), s=sigma), 0.05)

unittest.main(argv=['_'], exit=False);

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [8]:
PARAMS = {
    'prior': {
        'eta': {},
        'alpha': {},
    },
}

PARAMS['prior']['eta']['mean'], PARAMS['prior']['eta']['standard_deviation'] = (
    compute_log_norm_prior(1./2., 30., 0.9))
PARAMS['prior']['eta']['alpha'], PARAMS['prior']['eta']['standard_deviation'] = (
    compute_log_norm_prior(1., 4., 0.9))

PARAMS

{'prior': {'alpha': {},
  'eta': {'alpha': 0.6931471805599454,
   'mean': 1.3540251005511053,
   'standard_deviation': 0.4214035639417993}}}

In [11]:
class WeibullModel:
    def __init__(self, params) -> None:
        pass
    
    def forward(self):
        pass

In [13]:
def model_fn(features, labels, mode, params, config):
    del features, labels, mode, config
    
    model = WeibullModel(params)    
    return model.forward()

In [14]:
graph = tf.Graph()

with graph.as_default():
    init_op = tf.group(tf.global_variables_initializer())
    tmp = model_fn(None, None, tf.estimator.ModeKeys.TRAIN, PARAMS, tf.estimator.RunConfig())    

with graph.as_default(), tf.Session() as sess:
    sess.run(init_op)

In [2]:
failure_stresses

,0,1,2,3,4,5,6,7,8,9,10,11,12
Length (mm),,,,,,,,,,,,,
1,2.247,2.640,2.842,2.908,3.099,3.126,3.245,3.328,3.355,3.383,3.572,3.581,3.681
10,1.901,2.132,2.203,2.228,2.257,2.350,2.361,2.396,2.397,2.445,2.454,2.454,2.474
20,1.312,1.314,1.479,1.552,1.700,1.803,1.861,1.865,1.944,1.958,1.966,1.997,2.006
50,1.339,1.434,1.549,1.574,1.589,1.613,1.746,1.753,1.764,1.807,1.812,1.840,1.852
